In [1]:
import sys
sys.path.append("/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/hfnet_reproduce/hfnet_silk/")
from model.dataset.hpatches import HpatchesDataset

In [2]:
from torch.utils.data import DataLoader

import numpy as np
import torch

from model.hfnet import HFNet

from evaluation.local_descriptor import evaluate

In [3]:
# Load dataset
dataset = HpatchesDataset(alteration='all', make_pairs=True, shuffle=True)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [4]:
EXPER_PATH = '/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/reproduce/model/'
MODEL_PATH = '/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/hfnet_reproduce/hfnet_silk/'
models = { 
    'silk':{
        'align_corners':'weight/align_corners = False/last_model.pth', 
    },
}

config_model= {
    'image_channels':1,
    # 'loss_weights': 'uncertainties',
    'loss_weights':{
        'global':1,
        'local':1,
        'detector':1
    },
    'local_head': {
        'scale':2,
        'descriptor_dim': 128,
        'detector_grid': 8,
        'input_channels': 96
    },
    'global_head': {
        'n_clusters': 32,
        'intermediate_proj': 0,
        'dimensionality_reduction': 4096
    }
}

config = {
        'local':{
            'nms_radius':4,
            'detector_threshold': 0.015,
            'num_keypoints': 1000,
        },
        'do_ratio_test': True,
        'correct_match_thresh': 3,
        'correct_H_thresh': 3,
        'num_features': 300,
        # 'do_nms': True,
        # 'nms_thresh': 4,
        # 'remove_borders': 4,
    }
    


model = HFNet(config_model, width_mult=0.75)

In [5]:
import time

device = torch.device('cpu')
model.to(device)

for k,v in models.items():
    for kk,vv in v.items():
        print(f"{k}: {kk}")
        print(config)
        state_dict = torch.load(MODEL_PATH + vv, 
                                weights_only= False,
                                map_location=device)
        state_dict.pop("logvars", None) 
        model.load_state_dict(state_dict, strict=False)
        metrics,_,_=evaluate(model, dataloader, config)
        
        

        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(current_time)

        log_file = open('descriptors.txt', 'a')
        log_file.write('\n[{}]'.format(current_time))
        log_file.write('\n{}: {}'.format(k, kk))
        log_file.write('{}\n'.format(config))


        for km, vm in metrics.items():
                print('{:<25} {:.3f}'.format(km, vm))

                #save in log file
                log_file.write('{:<25} :{:.3f}  '.format(km, vm))
                log_file.write('\n')
        log_file.flush()
        print("-----------------------------------")

       

silk: align_corners
{'local': {'nms_radius': 4, 'detector_threshold': 0.015, 'num_keypoints': 1000}, 'do_ratio_test': True, 'correct_match_thresh': 3, 'correct_H_thresh': 3, 'num_features': 300}


100%|██████████| 580/580 [12:30<00:00,  1.29s/it]

2025-04-03 10:08:34
average_num_keypoints     300.000
matching_score            0.054
pose_correctness          0.079
mAP                       0.096
-----------------------------------
